### ETL: Primera parte.

```fix
Requisitos:
Datos:     Carpeta Data, con los cuatro csv de nuestros cuatro proveedores(netflix,amazon,disney,hulu).
           Carpeta ratings, con los 8 archivos csv.
Librerías: Pandas y Numpy. # pip install
```

La carpeta del proyecto está estructurada de la siguiente manera:
> ProjectML_OPS
 - Data
 - ProjectML_OPS_ML
 - ProjectML_OPS_API
 - ETL.ipynb (el presente archivo)
 - EDA.ipynb (siguiente archivo)

In [121]:
#Como primer paso, importamos las librerías necesarias. 

import pandas as pd
import numpy as np


In [122]:

# Leemos los archivos que están en la carpeta Data.

df_netflix = pd.read_csv("./Data/netflix_titles.csv", sep=",")
df_amazon = pd.read_csv("./Data/amazon_prime_titles.csv", sep=",")
df_disney = pd.read_csv("./Data/disney_plus_titles.csv", sep=",")
df_hulu = pd.read_csv("./Data/hulu_titles.csv", sep=",")


In [123]:
df_hulu.head()
df_hulu.shape

(3073, 12)

Las indicaciones a seguir son:

-Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

-Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

-De haber fechas, deberán tener el formato AAAA-mm-dd

-Los campos de texto deberán estar en minúsculas, sin excepciones

-El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [124]:
# Generamos el campo id, con la primera letra de la plataforma y el id ya existente de cada fila.
df_netflix['id'] = 'n' + df_netflix['show_id']
df_amazon['id'] = 'a' + df_amazon['show_id']
df_disney['id'] = 'd' + df_disney['show_id']
df_hulu['id'] = 'h' + df_hulu['show_id']
#Visualizamos
df_netflix.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",ns2
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,ns3
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",ns4
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,ns5


In [125]:
#Creamos una nueva tabla que tendra todos los datos. 
dataframes = [df_netflix, df_amazon, df_hulu, df_disney]
df =  pd.concat(dataframes)
# Visualizamos el tamaño del dataframe completo.
df.shape

(22998, 13)

Nos quedamos con un dataframe de 22998 filas × 13 columnas.

Ahora trabajamos las tablas de los ratings de usuarios.

In [126]:
#Leemos las tablas.
df_1 = pd.read_csv("./Data/Ratings/1.csv", sep=",")
df_2 = pd.read_csv("./Data/Ratings/2.csv", sep=",")
df_3 = pd.read_csv("./Data/Ratings/3.csv", sep=",")
df_4 = pd.read_csv("./Data/Ratings/4.csv", sep=",")
df_5 = pd.read_csv("./Data/Ratings/5.csv", sep=",")
df_6 = pd.read_csv("./Data/Ratings/6.csv", sep=",")
df_7 = pd.read_csv("./Data/Ratings/7.csv", sep=",")
df_8 = pd.read_csv("./Data/Ratings/8.csv", sep=",")

In [127]:
#Juntamos todas las tablas
datausers = [df_1, df_2, df_3, df_4,df_5,df_6,df_7, df_8]
df_users =  pd.concat(datausers)
df_users.head()

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [128]:
#Verificamos la columna rating antes de trabajarla.
df_users['rating'].isna().sum()

0

No tiene nulos. Podemos crear la columna 'score' dentro de la tabla, que luego unificaremos con la primera tabla.

In [129]:
df_users['score'] = df_users['rating'].astype(float)
#Usamos el promedio de las calificaciones de los usuarios para cada película, los agrupamos por el Id de la pelicula. 
df_users['score'] = df_users.groupby(['movieId'])['rating'].transform('mean')
#Verificamos la tabla
df_users.tail()

,userId,rating,timestamp,movieId,score
1499995,124380,4.5,1196786159,ns5272,3.593275
1499996,124380,2.5,1196786030,ns5492,3.536561
1499997,124380,3.5,1196785679,hs305,3.454183
1499998,124380,4.5,1196787089,ns7881,3.620278
1499999,124380,1.5,1196785847,as883,3.565476


In [130]:
#Con estas dos tablas realizamos un merge, considerando el campo movieId. 
#Antes, verificamos los nulos y valores en las columnas de ambas tablas. 
df['id'].isna().sum()
#0 
df_users['movieId'].isna().sum()
#0

0

In [131]:
#Convertimos ambos datos a string, para cercionarnos de que son del mismo tipo (condicion para poder hacer el merge)
df['id'] = df['id'].astype(str)
#Renombranos la columna movieId de df_users.
df_users.rename(columns = {'movieId':'id'}, inplace = True)
df_users['id'] = df_users['id'].astype(str)


In [132]:
#Realizamos el merge
dataset = pd.merge(df,df_users[['id','userId','rating','timestamp','score']], on='id', how='left')


In [133]:
dataset.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating_x,duration,listed_in,description,id,userId,rating_y,timestamp,score
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1,64,4.5,1085081152,3.611111
1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1,189,3.5,1281649224,3.611111
2,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1,212,2.5,1362774658,3.611111
3,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1,261,4.0,864375361,3.611111
4,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",ns1,303,3.5,1059692545,3.611111


In [134]:
dataset.shape

(11024289, 17)

Ahora trabajaremos con esta tabla.
Reemplazamos los valores nulos del campo rating por el string G.

In [135]:
#Reemplazamos los valores nulos del campo rating por el string G. Pero antes verificamos la columna.
dataset['rating_x'].value_counts()


TV-MA      1763550
TV-14      1505382
R          1030752
13+        1013673
TV-PG       792925
            ...   
57 min         456
45 min         455
136 min        455
130 min        450
121 min        447
Name: rating_x, Length: 105, dtype: int64

In [136]:
#Visualizamos que existen valores de minutos en la columna, ubicados inadecuadamente. Los cambiamos a donde debe.
dataset['duration'] = np.where(dataset['rating_x'].str.contains('min'), dataset['rating_x'], dataset['duration'])
dataset['duration'] = np.where(dataset['rating_x'].str.contains('Season'), dataset['rating_x'], dataset['duration'])


In [137]:
# Y lleno esos valores de rating, como nulos.
dataset['rating_x'] = np.where(dataset['rating_x'].str.contains('min'), np.nan, dataset['rating_x'])
dataset['rating_x'] = np.where(dataset['rating_x'].str.contains('Season'), np.nan, dataset['rating_x'])


In [138]:
dataset['rating_x'].value_counts()

TV-MA        1763550
TV-14        1505382
R            1030752
13+          1013673
TV-PG         792925
16+           740443
ALL           607576
18+           595262
PG-13         533194
PG            423771
TV-G          367286
TV-Y7         263114
TV-Y          221399
G             193933
7+            185105
NR            145527
TV-NR          50608
UNRATED        15758
TV-Y7-FV        9244
NC-17           2861
AGES_18_        1492
UR              1479
NOT_RATE        1404
AGES_16_         968
ALL_AGES         500
16               486
NOT RATED        478
Name: rating_x, dtype: int64

In [139]:

#Vemos que existen valores NOT_RATED,NOT_RATE, UNRATED. Unificamos todos a 'Unrated'
dataset['rating_x'].replace('UNRATED','Unrated',inplace= True)
dataset['rating_x'].replace('NOT_RATE','Unrated',inplace= True )
dataset['rating_x'].replace('NOT_RATED','Unrated',inplace= True)
dataset['rating_x'].replace('NOT RATED','Unrated',inplace= True)
dataset['rating_x'].replace('UR','Unrated',inplace= True)
dataset['rating_x'].replace('NR','Unrated',inplace= True)


In [140]:

#Ahora si, llenamos los nulos con G.
dataset['rating_x'].fillna('G',inplace= True)

In [141]:
#Verificamos que ya no hayan nulos.

dataset['rating_x'].isna().sum()

0

Las fechas deberán tener el formato AAAA-mm-dd.

In [142]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 11024288
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   show_id       object 
 1   type          object 
 2   title         object 
 3   director      object 
 4   cast          object 
 5   country       object 
 6   date_added    object 
 7   release_year  int64  
 8   rating_x      object 
 9   duration      object 
 10  listed_in     object 
 11  description   object 
 12  id            object 
 13  userId        int64  
 14  rating_y      float64
 15  timestamp     int64  
 16  score         float64
dtypes: float64(2), int64(3), object(12)
memory usage: 1.5+ GB


In [143]:
#Vemos que los campos son date_added y timestamp

dataset['date_added'] = pd.to_datetime(dataset['date_added'])
dataset['date_added'] = dataset['date_added'].dt.strftime('%Y-%m-%d')


In [144]:
dataset['timestamp2'] = pd.to_datetime(dataset['timestamp'], unit="s")
dataset['timestamp2']

0          2004-05-20 19:25:52
1          2010-08-12 21:40:24
2          2013-03-08 20:30:58
3          1997-05-23 08:16:01
4          2003-07-31 23:02:25
                   ...        
11024284   2013-10-19 05:38:17
11024285   2004-11-10 03:54:53
11024286   2009-02-21 14:00:17
11024287   2017-07-14 18:14:59
11024288   2005-01-31 14:36:14
Name: timestamp2, Length: 11024289, dtype: datetime64[ns]

In [145]:
dataset['timestamp2'] = dataset['timestamp2'].dt.strftime('%Y-%m-%d')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 11024288
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   show_id       object 
 1   type          object 
 2   title         object 
 3   director      object 
 4   cast          object 
 5   country       object 
 6   date_added    object 
 7   release_year  int64  
 8   rating_x      object 
 9   duration      object 
 10  listed_in     object 
 11  description   object 
 12  id            object 
 13  userId        int64  
 14  rating_y      float64
 15  timestamp     int64  
 16  score         float64
 17  timestamp2    object 
dtypes: float64(2), int64(3), object(13)
memory usage: 1.6+ GB


In [146]:
dataset['timestamp2'].value_counts()

2000-11-20    40265
2005-03-22    33342
1999-12-11    30459
2005-03-23    25113
1999-12-13    24077
              ...  
1998-10-28        2
1996-03-24        1
1996-03-23        1
1999-05-12        1
1999-06-06        1
Name: timestamp2, Length: 7751, dtype: int64

In [147]:
# Dropeamos el primer timestamp, y renombramos al segundo.
dataset.drop('timestamp', axis = 1,inplace = True)
dataset.rename(columns = {'timestamp2':'timestamp'},inplace = True)

In [148]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 11024288
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   show_id       object 
 1   type          object 
 2   title         object 
 3   director      object 
 4   cast          object 
 5   country       object 
 6   date_added    object 
 7   release_year  int64  
 8   rating_x      object 
 9   duration      object 
 10  listed_in     object 
 11  description   object 
 12  id            object 
 13  userId        int64  
 14  rating_y      float64
 15  score         float64
 16  timestamp     object 
dtypes: float64(2), int64(2), object(13)
memory usage: 1.5+ GB


Los campos de texto deberán estar en minúsculas, sin excepciones.

In [149]:
dataset['type'] = dataset['type'].str.lower()
dataset['title'] = dataset['title'].str.lower()
dataset['director'] = dataset['director'].str.lower()
dataset['cast'] = dataset['cast'].str.lower()
dataset['listed_in'] = dataset['listed_in'].str.lower()
dataset['country'] = dataset['country'].str.lower()
dataset['rating_x'] = dataset['rating_x'].str.lower()
dataset['duration'] = dataset['duration'].str.lower()
dataset['description'] = dataset['description'].str.lower()
	


In [150]:
#Verificamos el campo actor
dataset['cast']

0                     NaN
1                     NaN
2                     NaN
3                     NaN
4                     NaN
                ...      
11024284    charlie tahan
11024285    charlie tahan
11024286    charlie tahan
11024287    charlie tahan
11024288    charlie tahan
Name: cast, Length: 11024289, dtype: object

In [151]:
#Vemos que hay nulos, los llenamos con 'Sin Dato'.
dataset['cast'].fillna('Sin Dato',inplace= True)
dataset['cast']

0                Sin Dato
1                Sin Dato
2                Sin Dato
3                Sin Dato
4                Sin Dato
                ...      
11024284    charlie tahan
11024285    charlie tahan
11024286    charlie tahan
11024287    charlie tahan
11024288    charlie tahan
Name: cast, Length: 11024289, dtype: object

El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [152]:
#dataset['duration'].fillna('Sin Dato',inplace = True)

In [153]:
#Extraemos el duration int, del duration.
dataset['duration_int'] = dataset['duration'].str.extract('(\d+)')


In [154]:
#Convertimos a integer el duration int, pero antes llenamos los nulos con 0
dataset['duration_int'].replace(np.nan,0, inplace= True)
#Convertimos a int.
dataset['duration_int'] = dataset['duration_int'].astype(int) 


In [155]:
#Extraemos el duration type y lo convertimos a string
dataset['duration_type'] = dataset['duration'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()
dataset['duration_type'] = dataset['duration_type'].astype(str)


In [156]:
#Vemos la informacion de la columna duration_type.
dataset['duration_type'].value_counts()
#Vemos que hay min, season, seasons, y nan.


min        7529409
season     1934150
seasons    1107371
nan         453359
Name: duration_type, dtype: int64

In [157]:
#Reemplazamos los valores nan por 'min' si es pelicula, y 'season' si es serie. 
#dataset['duration_type'].replace('nan','Sin Dato', inplace = True)

dataset['duration_type'] = np.where((dataset['duration_type'] == 'nan') & (dataset['type'] == 'movie'),'min',dataset['duration_type'])
dataset['duration_type'] = np.where((dataset['duration_type'] == 'nan') & (dataset['type'] == 'tv show'),'season',dataset['duration_type'])

In [158]:
#Reemplazamos seasons por season.
dataset['duration_type'].replace('seasons','season', inplace = True)

In [159]:
#Verificamos
dataset['duration_type'].value_counts()

min       7867507
season    3156782
Name: duration_type, dtype: int64

In [160]:
#Antes de guardar el archivo, realizaremos una copia del dataset, en el que solo consideramos el score de cada pelicula. 
# El dataset más grande lo guardamos aparte.
dataset['score'].isna().sum()
#0

0

In [161]:
datasetp = dataset.copy()

In [162]:
# Verificamos la cantidad de filas
datasetp.shape

(11024289, 19)

In [163]:
#Como solo deseamos la columna score, dropeamos las demás.
datasetp.drop('userId', axis = 1,inplace = True)
datasetp.drop('rating_y', axis = 1,inplace = True)
datasetp.drop('timestamp',axis = 1,inplace = True)

In [164]:
#Dropeamos los duplicados. Tomamos en cuenta el campo id, el que es unico por plataforma.
datasetp.drop_duplicates(subset=['id'], keep='first', inplace = True, ignore_index=True)

In [165]:
datasetp.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating_x,duration,listed_in,description,id,score,duration_int,duration_type
0,s1,movie,dick johnson is dead,kirsten johnson,Sin Dato,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",ns1,3.611111,90,min
1,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",ns2,3.552632,2,season
2,s3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,ns3,3.597938,1,season
3,s4,tv show,jailbirds new orleans,NaN,Sin Dato,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",ns4,3.561616,1,season
4,s5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,ns5,3.593023,2,season


In [166]:
#Verificamos la cantidad de filas. Ahora tengo el mismo tamaño inicial en filas.
datasetp.shape

(22998, 16)

Guardamos el archivo pequeño para realizar el analisis exploratorio de datos, en nuestra carpeta de la API. Consideramos los Nans todavía.

In [168]:

datasetp.to_csv('./ProjectML_OPS_API/df.csv')


In [169]:
#Guardamos el archivo con ciertas columnas para realizar el machine learning, para que ocupe menos espacio. Usamos compresion gzip para que ocupe menos espacio.
dataset_f = dataset [["userId","rating_y","id",'title','type']]
dataset_f.to_csv('data.csv', compression='gzip')

In [ ]:
#Importante: Subimos el archivo a Deta Drive.

#Primero realizamos un pip install Deta, para manejar el drive. 
#Nos ubicamos en la carpeta donde tenemos nuestros archivos que queremos cargar.
from deta import Deta 
#Nos conectamos a Deta con nuestro data key.
deta = Deta("e0t3zaGELWHa_JjY9Bw5uJusD698dB1Qxqujkpo3h3SQy")
print(deta)
#Nos conectamos al drive llamado Data creado
drive = deta.Drive("data")
#Con esto subimos el dataset data.csv
drive.put('data.csv', path='./data.csv')


Tiempo de ejecución del notebook: Alrededor de 15 min.